In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load the data
data = pd.read_csv('continuous dataset.csv')

# Convert timestamp to datetime format
data['datetime'] = data['datetime'][:45261].apply(lambda x: datetime.strptime(x, '%d-%m-%Y %H:%M'))

# Set the datetime as index
data.set_index('datetime', inplace=True)

# Resample the data to hourly frequency
data = data.resample('H').mean()

# Create lag features
for i in range(1, 25):
    data['lag_{}'.format(i)] = data['nat_demand'].shift(i)

# Create rolling mean and standard deviation features
data['rolling_mean'] = data['nat_demand'].rolling(window=24).mean()
data['rolling_std'] = data['nat_demand'].rolling(window=24).std()

# Create weekday and hour features
data['weekday'] = data.index.weekday
data['hour'] = data.index.hour

# Remove missing values
data.dropna(inplace=True)

# Split the data into train and test sets
train_size = int(len(data) * 0.8)
train_data = data[:train_size]/1000
test_data = data[train_size:]/1000


In [2]:
#LSTM
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.losses import mean_squared_error

# Prepare the train and test data
X_train, y_train = train_data.drop('nat_demand', axis=1), train_data['nat_demand']
X_test, y_test = test_data.drop('nat_demand', axis=1), test_data['nat_demand']

# Reshape the data for LSTM
X_train = X_train.values.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test = X_test.values.reshape(X_test.shape[0], 1, X_test.shape[1])

# Define the model
model = Sequential()
model.add(LSTM(64, input_shape=(1, X_train.shape[2]), return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))

# Compile the model
model.compile(loss='mse', optimizer='adam')

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model
mae = history.history['val_loss'][-1]
mse = mean_squared_error(y_test, model.predict(X_test))
rmse = np.sqrt(mse)

ypred=model.predict(X_test)

Epoch 1/50
566/566 [==============================] - 6s 5ms/step - loss: 0.0407 - val_loss: 0.0044
Epoch 2/50
566/566 [==============================] - 2s 4ms/step - loss: 0.0034 - val_loss: 0.0028
Epoch 3/50
566/566 [==============================] - 2s 4ms/step - loss: 0.0023 - val_loss: 0.0019
Epoch 4/50
566/566 [==============================] - 2s 4ms/step - loss: 0.0019 - val_loss: 0.0024
Epoch 5/50
566/566 [==============================] - 2s 4ms/step - loss: 0.0016 - val_loss: 0.0019
Epoch 6/50
566/566 [==============================] - 3s 4ms/step - loss: 0.0014 - val_loss: 0.0013
Epoch 7/50
566/566 [==============================] - 2s 4ms/step - loss: 0.0014 - val_loss: 0.0012
Epoch 8/50
566/566 [==============================] - 2s 4ms/step - loss: 0.0013 - val_loss: 0.0012
Epoch 9/50
566/566 [==============================] - 3s 4ms/step - loss: 0.0013 - val_loss: 0.0025
Epoch 10/50
566/566 [==============================] - 2s 4ms/step - loss: 0.0013 - val_loss: 0.0013

In [11]:
#Convert Data to be 1-dimensional
ypred = ypred.ravel()
y_test = y_test.ravel()

In [12]:
#For MAE
mae=np.mean(np.abs(ypred-y_test))
print('MAE = {:.2f} (%)'.format(mae))

#For MAPE
def mean_absolute_percentage_error(y_test, ypred): 
    return np.mean(np.abs((y_test - ypred)/y_test))*100.
mape = mean_absolute_percentage_error(y_test, ypred)
print('MAPE = {:.2f} (%)'.format(mape))

# For RMSE
from sklearn.metrics import mean_squared_error 
mse=mean_squared_error(y_test,ypred) 
rsme=np.sqrt(mse) 
print('RSME = {:.2f} (%)'.format(rsme))

MAE = 0.02 (%)
MAPE = 1.58 (%)
RSME = 0.03 (%)
